# Обучение и скоринг)


In [2]:
# Import
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt 
import matplotlib.axes as axs
%matplotlib inline
plt.rc("font", size=14)
import seaborn as sns
sns.set()
sns.set(style="whitegrid", color_codes=True)

from sklearn.preprocessing import StandardScaler 
from sklearn.decomposition import PCA 
from sklearn.cluster import KMeans 


In [6]:
import warnings
warnings.filterwarnings('ignore')

In [7]:
# загрузка
purchase_df = pd.read_csv('purchase data.csv', index_col = 0)

# Scaler, PCA, K-Means
scaler = pickle.load(open('scaler.pickle', 'rb'))
pca = pickle.load(open('pca.pickle', 'rb'))
kmeans_pca = pickle.load(open('kmeans_pca.pickle', 'rb'))

# обработка фич
features = purchase_df[['Sex', 'Marital status', 'Age', 'Education', 'Income', 'Occupation', 'Settlement size']]
purchase_segm_std = scaler.transform(features)

# PCA
purchase_segm_pca = pca.transform(purchase_segm_std)

# сегментация
purchase_segm_kmeans_pca = kmeans_pca.predict(purchase_segm_pca)

# разработка лейблов для сегментов
purchase_predictors = purchase_df.copy()
purchase_predictors['Segment'] = purchase_segm_kmeans_pca
segment_dummies = pd.get_dummies(purchase_segm_kmeans_pca, prefix = 'Segment', prefix_sep = '_')
purchase_predictors = purchase_predictors.reset_index()
purchase_predictors = purchase_predictors.merge(segment_dummies, how='inner',
                                                left_on = purchase_predictors.index,
                                                right_on = segment_dummies.index)

In [8]:
df_pa = purchase_predictors.set_index('ID').iloc[:,1:].copy()
df_pa.head(5)

,Day,Incidence,Brand,Quantity,Last_Inc_Brand,Last_Inc_Quantity,Price_1,Price_2,Price_3,Price_4,...,Age,Education,Income,Occupation,Settlement size,Segment,Segment_0,Segment_1,Segment_2,Segment_3
ID,,,,,,,,,,,,,,,,,,,,,
200000001,1,0,0,0,0,0,1.59,1.87,2.01,2.09,...,47,1,110866,1,0,0,1,0,0,0
200000001,11,0,0,0,0,0,1.51,1.89,1.99,2.09,...,47,1,110866,1,0,0,1,0,0,0
200000001,12,0,0,0,0,0,1.51,1.89,1.99,2.09,...,47,1,110866,1,0,0,1,0,0,0
200000001,16,0,0,0,0,0,1.52,1.89,1.98,2.09,...,47,1,110866,1,0,0,1,0,0,0
200000001,18,0,0,0,0,0,1.52,1.89,1.99,2.09,...,47,1,110866,1,0,0,1,0,0,0


### Модель вероятности покупки 

In [15]:
# ЛогРег
from sklearn.linear_model import LogisticRegression 

In [16]:
# Разделяем выборку на X, y
output = df_pa['Incidence'] 
inputs = pd.DataFrame()
inputs['Mean_Price'] =df_pa.iloc[:, 6:11].mean(axis=1)

In [17]:
# Обучаем
Logitmodel_purchase = LogisticRegression(solver = 'sag')
Logitmodel_purchase.fit(inputs, output)

LogisticRegression(solver='sag')

In [18]:
Logitmodel_purchase.coef_

array([[-2.3484298]])

In [19]:
Logitmodel_purchase.predict_proba(inputs)[:, 1]

array([0.22914471, 0.23584929, 0.23584929, ..., 0.21218897, 0.20984308,
       0.20984308])